## Preparando o ambiente

In [1]:
# Carregando a biblioteca tidyverse
library(tidyverse) # Pacote com várias utilidades para analisar os dados

# Carregando os arquivos do conjunto de dados da Fitbit (alvo da análise)
list.files(path = "../input")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     


── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


[1] "fitbit"

## Preparando os dados

Para a análise das funcionalidades, as tabelas utilizadas serão as de: **atividade diária, frequência cardíaca e monitoramento de sono**. Portanto, é preciso criar um dataframe para cada uma dessas tabelas.

In [2]:
# Criando um dataframe da atividade diária
daily_activity <- read_csv("/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv")

# Remoção de linhas nulas
daily_activity <- daily_activity %>% drop_na()

# Pré-visualização desse dataframe
head(daily_activity)

Rows: 940 Columns: 15


── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): ActivityDate
dbl (14): Id, TotalSteps, TotalDistance, TrackerDistance, LoggedActivitiesDi...



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1503960366,4/12/2016,13162,8.50,8.50,0,1.88,0.55,6.06,0,25,13,328,728,1985
1503960366,4/13/2016,10735,6.97,6.97,0,1.57,0.69,4.71,0,21,19,217,776,1797
1503960366,4/14/2016,10460,6.74,6.74,0,2.44,0.40,3.91,0,30,11,181,1218,1776
1503960366,4/15/2016,9762,6.28,6.28,0,2.14,1.26,2.83,0,29,34,209,726,1745
1503960366,4/16/2016,12669,8.16,8.16,0,2.71,0.41,5.04,0,36,10,221,773,1863
1503960366,4/17/2016,9705,6.48,6.48,0,3.19,0.78,2.51,0,38,20,164,539,1728


Esse dataframe expressa as principais características de atividade diária dos usuários. Nessa tabela, os dados sobre a quantidade de passos, distância percorrida, intensidade de atividade e tempo em relação a cada intensidade são agrupados.

In [3]:
# Criando um dataframe para frequência cardíaca
heart_rate <- read_csv("/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/heartrate_seconds_merged.csv")

# Remoção de linhas nulas
heart_rate <- heart_rate %>% drop_na()

# Pré-visualização desse dataframe
head(heart_rate)

Rows: 2483658 Columns: 3


── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): Time
dbl (2): Id, Value



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Id,Time,Value
<dbl>,<chr>,<dbl>
2022484408,4/12/2016 7:21:00 AM,97
2022484408,4/12/2016 7:21:05 AM,102
2022484408,4/12/2016 7:21:10 AM,105
2022484408,4/12/2016 7:21:20 AM,103
2022484408,4/12/2016 7:21:25 AM,101
2022484408,4/12/2016 7:22:05 AM,95


In [4]:
# Descobrindo o último dia de medição da frequência cardíaca
heart_rate %>%
    summarize(ultimo_dia = max(Time))

ultimo_dia
<chr>
5/9/2016 9:59:59 PM


Esse dataframe expressa a frequência cardíaca de cada usuário de 5 em 5 segundos todos os dias, do dia 12/04/2016 até o dia 09/05/2016.

In [5]:
# Criando um dataframe para o monitoramento do sono
sleep_day <- read_csv("/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv")

# Remoção de linhas nulas
sleep_day <- sleep_day %>% drop_na()

# Pré-visualização desse dataframe
head(sleep_day)

Rows: 413 Columns: 5


── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): SleepDay
dbl (4): Id, TotalSleepRecords, TotalMinutesAsleep, TotalTimeInBed



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1503960366,4/12/2016 12:00:00 AM,1,327,346
1503960366,4/13/2016 12:00:00 AM,2,384,407
1503960366,4/15/2016 12:00:00 AM,1,412,442
1503960366,4/16/2016 12:00:00 AM,2,340,367
1503960366,4/17/2016 12:00:00 AM,1,700,712
1503960366,4/19/2016 12:00:00 AM,1,304,320


In [6]:
# Descobrindo o último dia do monitoramento de sono
sleep_day %>%
    summarize(ultimo_dia_sono = max(SleepDay))

ultimo_dia_sono
<chr>
5/9/2016 12:00:00 AM


Esse dataframe expressa quantos sonos foram gravados, a quantidade de minutos dormidos e minutos na cama de cada usuário, do dia 12/04/2016 até o dia 09/05/2016.